# 使用 GRPO 训练模型

## GRPO 原理

GRPO（Group Relative Policy Optimization，群体相对策略优化）是一种由DeepSeek团队提出的强化学习算法，旨在优化大语言模型（LLM）的推理能力。与传统PPO算法依赖价值函数评估动作优劣不同，GRPO通过生成多个候选答案构成“群体”，利用群体内奖励的相对差异（如归一化奖励或排名优势）计算优势函数，从而避免维护额外的价值网络，显著降低显存占用和计算成本。其核心创新包括：

 1. **分组采样机制**，同一问题生成多组输出以评估相对质量；
 2. **群体竞争优化**，通过奖励均值和标准差计算相对优势，引导模型强化优质答案；
 3. **KL散度约束**，直接控制策略更新幅度以提升训练稳定性。

此外，GRPO兼容LoRA等高效微调技术，可将显存需求降低至7GB，使消费级GPU也能支持复杂推理模型的训练，推动了AI技术平民化。该算法已在数学推理、代码生成等任务中验证了性能优势，并被迁移至视觉语言模型领域，展现了广泛的应用潜力。其原理图如下所示：

![GRPO Struct](img/GRPO-Struct.svg)

具体公式与原理参考 Deepseek-R1 论文第 2.2 章：[Github - DeepSeek_R1.pdf](https://github.com/deepseek-ai/DeepSeek-R1/blob/main/DeepSeek_R1.pdf)

## GRPO 训练

### 1. 引入必要的库

In [ ]:
from datasets import load_dataset,Dataset
from trl import GRPOConfig, GRPOTrainer

### 2. 加载并设计训练数据

这里我们定义了一个标签 `<answer></answer>` 我们期望训练的模型输出可以用该标签包裹，这里就是将训练集设置为指定的格式：

In [ ]:
data_path = "./dataset/tldr"

answer_label = 'answer'
SYSTEM_PROMPT = f'你的回答需要在<{answer_label}></{answer_label}>标签内。'
XML_COT_FORMAT = f'<{answer_label}>' + '{answer}' + f'</{answer_label}>'

# 数据处理，将 assistant 的输出处理为指定格式，即用 <answer>...</answer> 的格式
def get_dataset() -> Dataset:

    data = Dataset.load_from_disk(data_path)
    data = data.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            # few shot, 因为0.5B模型太弱了
            {'role': 'user', 'content': '数字10203040里面有几个0?'},
            {'role': 'assistant', 'content': XML_COT_FORMAT.format(answer='4个0')},
            {'role': 'user', 'content': x['prompt']}
        ],
        'answer': x['completion']
    })
    return data

dataset = get_dataset()
dataset = dataset.remove_columns("completion")

**检查数据内容：**

In [ ]:
print(dataset[0])

### 3. 设计奖励函数（核心）

这里的奖励函数需要根据具体的需求进行设计，具体可以参考下面的奖励函数：

In [ ]:
import re

def extract_answer(completion):
    """
    提取答案

    :param completion: 补全的内容
    :return: 在指定标签内的内容
    """
    pattern = f'^.*<{answer_label}>(.+)</{answer_label}>.*$'
    match=re.search(pattern,completion,re.DOTALL)
    if match:
        answer=match.group(1)
    else:
        answer=None
    return answer

# 内容奖励
def reward_content(completions, answer, **kwargs):
    """
    补全的内容比回答的内容长就奖励
    """
    scores = []
    for idx,completion in enumerate(completions):

        response_answer = extract_answer(completion[0]['content'])
        if response_answer is None:
            scores.append(0)
            continue

        dlen = len(answer[idx]) - len(response_answer)
        if dlen > 0:
            scores.append(5)
        else:
            scores.append(0)
    return scores

# 宽松标签奖励
def reward_label(completions, **kwargs):
    """
    只要有 <answer> 标签就奖励
    """
    print(completions)

    pattern = f'^.*<{answer_label}>(.+)</{answer_label}>.*$'
    scores = []
    for completion in completions:
        if re.fullmatch(pattern, completion[0]['content']):
            scores.append(5)
        else:
            scores.append(0)
    return scores

# 严格标签奖励
def reward_label_strict(completions, **kwargs):
    """
    必须按照 <answer> ... </answer> 的格式输出，才有奖励
    """
    pattern = f'^<{answer_label}>(.+)</{answer_label}>$'
    scores = []
    for completion in completions:
        if re.fullmatch(pattern, completion[0]['content']):
            scores.append(10)
        else:
            scores.append(0)
    return scores

### 4. 进行训练

GRPO 训练器配置参数 `GRPOConfig` 继承自 `TrainingArguments` ，下面是其相关配置：：

| 分类            | 参数名称                          | 类型/选项                     | 默认值      | 说明                                                               |
|---------------|-------------------------------|---------------------------|----------|------------------------------------------------------------------|
| **模型和参考模型**   | `model_init_kwargs`           | `dict[str, Any]` 或 `None` | `None`   | 用于 [`~transformers.AutoModelForCausalLM.from_pretrained`] 的关键字参数 |
| **数据预处理**     | `remove_unused_columns`       | `bool`                    | `False`  | 是否仅保留数据集中的 `"prompt"` 列                                          |
|               | `max_prompt_length`           | `int` 或 `None`            | `512`    | 提示词最大长度（超长时左截断）                                                  |
|               | `num_generations`             | `int` 或 `None`            | `8`      | 每个提示词的生成次数（全局批次大小必须可被此值整除）                                       |
|               | `temperature`                 | `float`                   | `0.9`    | 采样温度值（越高随机性越强）                                                   |
|               | `max_completion_length`       | `int` 或 `None`            | `256`    | 生成结果的最大长度                                                        |
|               | `ds3_gather_for_generation`   | `bool`                    | `True`   | DeepSpeed ZeRO-3 下是否聚合权重加速生成（禁用可训练超显存模型但速度降低）                    |
| **vLLM 生成加速** | `use_vllm`                    | `bool`                    | `False`  | 是否使用 vLLM 加速生成（需单独保留 GPU）                                        |
|               | `vllm_device`                 | `str`                     | `"auto"` | vLLM 生成设备（自动选择可用 GPU）                                            |
|               | `vllm_gpu_memory_utilization` | `float`                   | `0.9`    | vLLM GPU 内存利用率（高值提升吞吐但可能 OOM）                                    |
|               | `vllm_dtype`                  | `str`                     | `"auto"` | vLLM 生成数据类型（自动根据模型配置选择）                                          |
|               | `vllm_max_model_len`          | `int` 或 `None`            | `None`   | vLLM 最大模型长度（优化显存不足时的 KV 缓存效率）                                    |
| **训练控制**      | `learning_rate`               | `float`                   | `1e-6`   | AdamW 优化器的初始学习率（覆盖父类默认值）                                         |
|               | `beta`                        | `float`                   | `0.04`   | KL 散度系数                                                          |
|               | `reward_weights`              | `list[float]` 或 `None`    | `None`   | 奖励函数权重列表（未设置时等权 1.0）                                             |
|               | `sync_ref_model`              | `bool`                    | `False`  | 是否定期同步参考模型（需配合 TR-DPO 参数）                                        |
|               | `ref_model_mixup_alpha`       | `float`                   | `0.9`    | TR-DPO 混合系数（π_ref = απ_θ + (1-α)π_ref_prev）                      |
|               | `ref_model_sync_steps`        | `int`                     | `64`     | TR-DPO 参考模型同步频率                                                  |
| **日志控制**      | `log_completions`             | `bool`                    | `False`  | 是否在训练期间记录生成结果                                                    |

注意，推荐使用 vllm 进行加速，即令 `use_vllm = True` ，但如果在 windows 系统下直接安装 vllm 可能会出现一些问题，请参考下面方案进行安装 ：[vllm在Windows系统下的安装解决方案](docs/vllm在Windows系统下的安装解决方案.md)

In [ ]:
model_path = "E:/AI/Models/Qwen/Qwen2.5-0.5B"
output_dir = "./output/grpo"

training_args = GRPOConfig(
    output_dir = output_dir,

    num_train_epochs = 1,            # 训练轮数
    learning_rate = 5e-6,            # 学习率
    lr_scheduler_type = "cosine",    # 学习率调度器的类型，这里采用余弦函数衰减
    adam_beta1 = 0.9,                # AdamW 一阶矩估计的衰减率 beta1
    adam_beta2 = 0.99,               # AdamW 二阶矩估计的衰减率 beta2
    weight_decay = 0.1,              # 权重衰减（L2 正则化），防止模型过拟合，通过在损失函数中添加权重的 L2 范数来约束模型参数。

    per_device_train_batch_size = 2, # 每个设备训练的批大小
    gradient_accumulation_steps = 4, # 累积梯度，等效 batch_size=2*4=8

    bf16 = True,                     # 使用混合精度

    num_generations = 2,             # 每个提示词生成的次数
    max_prompt_length = 256,         # 最大提示词长度
    max_completion_length = 300,     # 最大补全长度
    save_steps = 100,
    logging_steps=10,
    max_grad_norm = 0.1,             # 梯度裁剪的最大范数，防止梯度爆炸，通过裁剪梯度的范数使其不超过设定值。
    warmup_ratio = 0.1,              # 学习率预热的比例。
    # report_to = "tensorboard",
)

trainer = GRPOTrainer(
    model=model_path,
    reward_funcs=[
        reward_label,
        reward_label_strict,
        reward_content
    ],
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

### 5. 保存模型

In [ ]:
# 保存路径
output_dir = "./output/final"

trainer.model.save_pretrained(output_dir)